In [6]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time

# Configuração inicial do WebDriver
service = Service()
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920x1080")
options.add_argument("--disable-notifications")
options.add_argument('--no-sandbox')
options.add_argument('--verbose')
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

# Inicie o navegador
browser = webdriver.Chrome(service=service, options=options)

# Lista de URLs das ações
acoes_urls = ['https://investidor10.com.br/acoes/taee11/', 'https://investidor10.com.br/acoes/wege3/', 'https://investidor10.com.br/acoes/petr4/']

df_final = pd.DataFrame()

for url in acoes_urls:
    browser.get(url)
    time.sleep(10)
    page_content = browser.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    table = soup.find(id='table-indicators-history')

    if table:
        years = [th.get_text().strip() for th in table.find_all("th", class_="year")]
        indicadores = [td.get_text().strip() for td in table.find_all("td", class_="indicator")]

        all_values = []
        for row in table.find_all("tr", class_=["visible-even", "visible-odd"]):
            values = [value_td.get_text().strip() for value_td in row.find_all("td", class_="value")]
            if len(values) == len(years):
                all_values.append(values)

        df_temp = pd.DataFrame(all_values, columns=years, index=indicadores)
        df_temp['acao'] = url[33:].strip("/")   # Adiciona uma coluna para identificar a ação
        df_final = df_final.append(df_temp)

browser.quit()
print(df_final)

In [8]:
df_final.to_csv("investidor10_acoes.csv")